In [14]:
from langchain_huggingface import HuggingFaceEndpoint  # Import Hugging Face endpoint class
from secret_api_keys import hugging_face_api_key # Import secret API key from separate file
from langchain.prompts import PromptTemplate  # Import PromptTemplate class from langchain
import os  # Import the 'os' module for potential system interactions
import re  # Import the 're' module for regular expressions

In [15]:
# Set the Hugging Face Hub API token as an environment variable
os.environ['HUGGINGFACEHUB_API_TOKEN'] = hugging_face_api_key

In [16]:
# Define the Hugging Face model repository ID
repo_id = "meta-llama/Meta-Llama-3-8B-Instruct"
# Create a Hugging Face Endpoint instance
llm = HuggingFaceEndpoint(
    repo_id=repo_id,  # Specify the model repository ID
    temperature=0.6,  # Set the temperature parameter (controls randomness)
    token=hugging_face_api_key,  # Use the API key for authentication
)


WARNING! token is not default parameter.
                    token was transferred to model_kwargs.
                    Please make sure that token is what you intended.


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to C:\Users\balup\.cache\huggingface\token
Login successful


In [17]:
print(llm('I want to write a blog related to on machine learning suggest me some titles for the blog.'))

 

Here are a few title suggestions for your machine learning blog:

1. **"Machine Learning Mastery: Tips and Tricks for Beginners"**: This title suggests that your blog will provide guidance and expertise for those new to machine learning.
2. **"The Machine Learning Chronicles: Stories of Success and Failure"**: This title implies that your blog will share personal anecdotes and experiences with machine learning, which can be engaging and relatable.
3. **"Machine Learning Uncovered: Demystifying the Technology"**: This title suggests that your blog will provide a deep dive into the inner workings of machine learning, making it accessible to a wider audience.
4. **"The Machine Learning Journey: From Idea to Implementation"**: This title implies that your blog will take readers through the entire process of machine learning, from conceptualization to deployment.
5. **"Machine Learning Insights: Trends, Tools, and Techniques"**: This title suggests that your blog will provide expert anal

In [18]:
# Define a PromptTemplate for title suggestions
prompt_template_for_title_suggestion = PromptTemplate(
    input_variables=['topic'],  # Specify input variables
    template =  # Define the prompt template
    '''
    I'm planning a blog post on topic : {topic}.
    The title is informative, or humorous, or persuasive. 
    The target audience is beginners, tech enthusiasts.  
    Suggest a list of ten creative and attention-grabbing titles for this blog post. 
    Don't give any explanation or overview to each title.
    '''
)


In [19]:
formatted_prompt = prompt_template_for_title_suggestion.format(topic='basket ball')
print(formatted_prompt)


    I'm planning a blog post on topic : basket ball.
    The title is informative, or humorous, or persuasive. 
    The target audience is beginners, tech enthusiasts.  
    Suggest a list of ten creative and attention-grabbing titles for this blog post. 
    Don't give any explanation or overview to each title.
    


In [20]:
title_suggestion_chain = prompt_template_for_title_suggestion | llm # defining the title suggestion chain

In [21]:
topic_name = 'basket ball'
print(title_suggestion_chain.invoke({topic_name})) # Generate titles using the title suggestion chain





1. "The Slam Dunk Guide to Basketball for Beginners"
2. "Hoops and Bytes: The Ultimate Guide to Basketball for Tech Enthusiasts"
3. "From Zero to Hero: Mastering Basketball in 10 Easy Steps"
4. "The Basketball Bible: A Beginner's Guide to the Game"
5. "Dribble, Pass, Score: A Beginner's Guide to Basketball"
6. "The Tech-Savvy Basketball Player: Tips and Tricks for Success"
7. "The Ultimate Basketball Experience: A Beginner's Guide"
8. "The Basketball Blueprint: A Step-by-Step Guide to Mastering the Game"
9. "Get Ready to Run the Court: A Beginner's Guide to Basketball"
10. "The Basketball Revolution: How Technology is Changing the Game"


In [22]:
# Define a PromptTemplate for blog content generation
prompt_template_for_title = PromptTemplate(
    input_variables=['title', 'keywords', 'blog_length'],  # Specify input variables
    template=  # Define the prompt template
    '''Write a high-quality, informative, and plagiarism-free blog post on the topic: "{title}". 
    Target the content towards a beginner audience. 
    Use a conversational writing style and structure the content with an introduction, body paragraphs, and a conclusion. 
    Try to incorporate these keywords: {keywords}. 
    Aim for a content length of {blog_length} words. 
    Make the content engaging and capture the reader's attention.'''
)

In [23]:
title_chain = prompt_template_for_title | llm # Create a chain for title generation

In [24]:
title_of_the_blog = 'The Court is Your Canvas: How Basketball Can Improve Your Life'
num_of_words = 200
formatted_keywords = 'nike, tall, annual revenue, fan base, world record'

In [25]:
print(title_chain.invoke({'title': title_of_the_blog, 'keywords': formatted_keywords, 'blog_length':num_of_words}))

 

Here is a blog post that meets your requirements:

---

The Court is Your Canvas: How Basketball Can Improve Your Life

As the iconic Nike slogan goes, "Just Do It." And what better way to do it than on the basketball court? Basketball is more than just a game - it's a way to challenge yourself, build relationships, and improve your overall well-being. Whether you're a seasoned pro or a beginner, the court is your canvas, waiting for you to create your masterpiece.

As a tall player, you may have an advantage on the court, but it's not just about physical attributes. Basketball requires strategy, teamwork, and mental toughness. By playing the game, you'll develop these skills, which can translate to other areas of your life. Plus, with an annual revenue of over $20 billion, the basketball industry is a global powerhouse, offering endless opportunities for growth and development.

As a fan of the game, you're part of a massive fan base that spans the globe. Basketball has the power t

In [ ]:
# input_topic = 'machine translation'
titles_output = chain.invoke({'machine'})

In [ ]:
sentences = titles_output.split('\n')
corrected_sentences = [sentence.strip().lstrip('0123456789.').strip().strip('"') for sentence in sentences if len(sentence.strip().lstrip('0123456789.').strip().strip('"')) > 0]

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
similarity_model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')

# Tokenize sentences
encoded_input = tokenizer(corrected_sentences, padding=True, truncation=True, return_tensors='pt')

# Compute token embeddings
with torch.no_grad():
    model_output = similarity_model(**encoded_input)

# Perform pooling
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

# Normalize embeddings
sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)

print("Sentence embeddings:")
print(sentence_embeddings)


Sentence embeddings:
tensor([[-0.1106, -0.0063, -0.0156,  ...,  0.0368,  0.0605, -0.0157],
        [-0.0360,  0.0419,  0.0446,  ..., -0.0180,  0.0430, -0.0429],
        [-0.0576,  0.0080, -0.0399,  ...,  0.0136,  0.1202, -0.0483],
        ...,
        [-0.0345,  0.0690,  0.0307,  ...,  0.0030,  0.0398,  0.0889],
        [-0.0090,  0.0789, -0.0172,  ...,  0.0263, -0.0092,  0.0094],
        [-0.0445, -0.0679, -0.0217,  ...,  0.0155, -0.0671,  0.0235]])


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

similarity_dict = {
    corrected_sentences[i]: cosine_similarity(
        sentence_embeddings[i].reshape(1, -1),  # Reshape to 2D
        sentence_embeddings[0].reshape(1, -1)
    )
    for i in range(1, len(sentence_embeddings))
}

In [ ]:
similarity_dict

{'The Impact of Machines on Human Life': array([[0.6505365]], dtype=float32),
 'The Role of Machines in Industry and Manufacturing': array([[0.61791193]], dtype=float32),
 'The Future of Machines: Trends and Predictions': array([[0.7966087]], dtype=float32),
 'How Machines are Revolutionizing the World of Work': array([[0.61681235]], dtype=float32),
 'Note: I assume that you want the titles to be general and not specific to any particular type of machine. If you want titles related to a specific type of machine, please let me know.': array([[0.39382726]], dtype=float32),
 'Please let me know if you want me to suggest more titles.': array([[0.04360505]], dtype=float32),
 'I can suggest more titles related to specific types of machines like robotics, artificial intelligence, computer vision, etc. Let me know if you are interested.': array([[0.4604401]], dtype=float32),
 "Also, if you want me to suggest titles related to a specific industry or sector like healthcare, finance, education, e

In [ ]:

# Create a list of tuples, where each tuple contains the key and its similarity value
data_list = [(key, value[0]) for key, value in similarity_dict.items()]

# Sort the list by the similarity value in descending order (highest similarity first)
data_list.sort(key=lambda x: x[1], reverse=True)

for i, j in data_list[:5]:
    print(i)
# # Create a new dictionary with the sorted order
# sorted_data = {key: value for key, value in data_list}

# # Print the sorted dictionary
# for key, value in sorted_data.items():
#     print(key, value)

The Future of Machines: Trends and Predictions
The Impact of Machines on Human Life
The Role of Machines in Industry and Manufacturing
How Machines are Revolutionizing the World of Work
I can suggest more titles related to specific types of machines like robotics, artificial intelligence, computer vision, etc. Let me know if you are interested.
